<a href="https://colab.research.google.com/github/SirwaniViren/MSc-Gousto-Project/blob/main/initial_framework_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import pandas as pd
import random
from collections import defaultdict
from random import randrange

In [2]:
random.seed(42)

In [ ]:
def create_eligibility_dict(num_factory, num_items):
    items = list(range(100, 100 + num_items * 10, 10))
    eligibility_dict = {}

    # Start with a small number of items for the first factory
    num_eligible_items = random.randint(1, 10)

    for i in range(1, num_factory + 1):
        if i == num_factory:
            # Last factory gets all items
            eligibility_dict[f"F{i}"] = items
        else:
            eligibility_dict[f"F{i}"] = items[:num_eligible_items]
            # Randomly determine the number of additional items for the next factory, allowing for fluctuations
            num_eligible_items += random.randint(-5, 15)
            # Ensure num_eligible_items is within bounds and at least 1
            num_eligible_items = max(1, min(num_eligible_items, num_items))

    return eligibility_dict

In [20]:
def create_factory_caps(num_factory, max_boxes):
  factory_caps = {"F" + str(i): randrange(1, max_boxes) for i in range(1, (num_factory + 1))}
  num_recipies = sum(factory_caps.values())
  return factory_caps, num_recipies

def generate_orders(num_recipies_ordered, num_items, max_items_per_order=5):
    items = list(range(100, 100 + num_items * 10, 10))
    orders = []
    order_id = 1

    while num_recipies_ordered > 0:
        num_items_in_order = min(random.randint(1, max_items_per_order), num_recipies_ordered)
        order_items = random.sample(items, num_items_in_order)
        for item in order_items:
            orders.append({'order_id': order_id, 'item_id': item})
        num_recipies_ordered -= num_items_in_order
        order_id += 1

    orders_df = pd.DataFrame(orders)
    return orders_df

In [ ]:
def compute_orders_eligibility(orders_df, eligibility_dict):
    orders_eligibility = []

    for order_id in orders_df['order_id'].unique():
        order_items = orders_df[orders_df['order_id'] == order_id]['item_id'].tolist()
        eligible_factories = []
        for factory, items in eligibility_dict.items():
            if all(item in items for item in order_items):
                eligible_factories.append(factory)
        orders_eligibility.append({'order_id': order_id, 'eligibility': eligible_factories})

    orders_eligibility_df = pd.DataFrame(orders_eligibility)
    return orders_eligibility_df

In [ ]:
factory_caps, num_recipies = create_factory_caps(20, 100)
orders = generate_orders(num_recipies, 100)
orders

In [ ]:
# Function to allocate items
def allocate_items_greedy(orders_df, factory_caps, eligibility_id):
  allocation = defaultdict(lambda: defaultdict(int))
  current_order_id = None
  current_items = []
  current_sites = []

  for row in orders_df.itertuples():
    order_id = row.order_id
    item_id = row.item_id
    site = row.site

    if current_order_id is None:
        current_order_id = order_id

    if order_id != current_order_id:
        # Allocate items for the previous order
        for s in current_sites:
            if all(item in eligibility_id[s] for item in current_items) and factory_caps[s] > 0:
                for item in current_items:
                    allocation[item][s] += 1
                factory_caps[s] -= 1
                break

        # Reset for the new order
        current_order_id = order_id
        current_items = []
        current_sites = []

    current_items.append(item_id)
    current_sites.append(site)

  # Allocate items for the last order
  for s in current_sites:
    if all(item in eligibility_id[s] for item in current_items) and factory_caps[s] > 0:
      for item in current_items:
        allocation[item][s] += 1
      factory_caps[s] -= 1
      break

  return allocation


# Allocate items for LD5
ld5_allocation = allocate_items_greedy(orders_to_recipe_ld5_df, factory_caps.copy(), eligibility_id)
ld0_allocation = allocate_items_greedy(orders_to_recipe_ld0_df, factory_caps.copy(), eligibility_id)

In [ ]:
def convert_dict_df(allocation_dict, alloc_day):
  # Convert allocation to dictionary with required format
  allocation_result = []
  for item_id, sites in allocation_dict.items():
      for site, count in sites.items():
          allocation_result.append({'item_id': item_id, 'site': site, alloc_day: count})

  return pd.DataFrame(allocation_result)

ld5_df = convert_dict_df(allocation_dict=ld5_allocation, alloc_day='lead_5')
ld0_df = convert_dict_df(allocation_dict=ld0_allocation, alloc_day='lead_0')

In [ ]:
def merge_allocation(ld5_df, ld0_df):
    # Merge the two dataframes on 'item_id' and 'site'
    merged_df = pd.merge(ld5_df, ld0_df, on=['item_id', 'site'], how='outer')

    # Fill NaN values with zeros
    merged_df.fillna(0, inplace=True)

    # Ensure the integer type for lead_5 and lead_0 columns
    merged_df['lead_5'] = merged_df['lead_5'].astype(int)
    merged_df['lead_0'] = merged_df['lead_0'].astype(int)

    return merged_df

# Merge the dataframes and calculate the absolute error
merged_df = merge_allocation(ld5_df, ld0_df)
total_forecast = merged_df["lead_5"].sum()

In [ ]:
print(merged_df)

    item_id site  lead_5  lead_0
0        10   F1       1       1
1        10   F2       1       1
2        10   F3       1       0
3        20   F1       1       0
4        20   F3       1       2
5        20   F2       1       0
6        50   F1       1       0
7        50   F3       1       1
8        30   F2       2       1
9        40   F3       1       0
10       30   F3       0       1
11       40   F1       0       1


In [ ]:
site_df = merged_df.copy()
site_df['abs_error'] = (site_df['lead_5'] - site_df['lead_0']).abs()
print(site_df)

# Calculate WMAPE_site
wmape_site = site_df["abs_error"].sum() / total_forecast

# Display WMAPE_site
print(f"\nWeighted Mean Absolute Percentage Error (WMAPE) Site: {wmape_site:.4f}")

    item_id site  lead_5  lead_0  abs_error
0        10   F1       1       1          0
1        10   F2       1       1          0
2        10   F3       1       0          1
3        20   F1       1       0          1
4        20   F3       1       2          1
5        20   F2       1       0          1
6        50   F1       1       0          1
7        50   F3       1       1          0
8        30   F2       2       1          1
9        40   F3       1       0          1
10       30   F3       0       1          1
11       40   F1       0       1          1

Weighted Mean Absolute Percentage Error (WMAPE) Site: 0.8182


In [ ]:
global_df = merged_df.copy()
global_df = global_df.groupby('item_id').sum().reset_index()
global_df['abs_error'] = (global_df['lead_5'] - global_df['lead_0']).abs()
global_df = global_df.drop(['site'], axis=1)
print(global_df)


# Calculate WMAPE_site
wmape_global = global_df["abs_error"].sum() / total_forecast

# Display WMAPE_site
print(f"\nWeighted Mean Absolute Percentage Error (WMAPE) Global: {wmape_global:.4f}")

   item_id  lead_5  lead_0  abs_error
0       10       3       2          1
1       20       3       2          1
2       30       2       2          0
3       40       1       1          0
4       50       2       1          1

Weighted Mean Absolute Percentage Error (WMAPE) Global: 0.2727
